In [1]:
import numpy as np
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk

In [2]:
dataset_df = pd.read_csv('php_xss_datasets.csv', encoding='utf-8')
opcode_list = np.array(dataset_df['opcode'].tolist())
label_list = np.array(dataset_df['label'].tolist())
print(opcode_list.shape)
print(label_list.shape)
print(opcode_list[0])

(10080,)
(10080,)
ECHO FETCH_R FETCH_DIM_R ASSIGN INIT_FCALL SEND_VAR SEND_VAL DO_ICALL ASSIGN INIT_FCALL SEND_VAR SEND_VAL DO_ICALL JMPZ ASSIGN JMP ASSIGN ECHO ECHO RETURN


In [3]:
corpus = []
for opcode in opcode_list:
    corpus.append(list(nltk.word_tokenize(opcode)))
corpus = np.array(corpus)
print(corpus.shape)




(10080,)


In [4]:
print(corpus[0])

['ECHO', 'FETCH_R', 'FETCH_DIM_R', 'ASSIGN', 'INIT_FCALL', 'SEND_VAR', 'SEND_VAL', 'DO_ICALL', 'ASSIGN', 'INIT_FCALL', 'SEND_VAR', 'SEND_VAL', 'DO_ICALL', 'JMPZ', 'ASSIGN', 'JMP', 'ASSIGN', 'ECHO', 'ECHO', 'RETURN']


In [4]:
tgd_dm = [TaggedDocument(doc, [i]) for doc, i in zip(corpus, range(label_list.shape[0]))]
print(len(tgd_dm))
doc_model = Doc2Vec(tgd_dm, vector_size=256, window=5, min_count=1, workers=4)

10080


In [5]:
print(len(doc_model.docvecs))
print(doc_model.docvecs[0].shape)
doc_vec_list = doc_model.docvecs.vectors_docs
print(doc_vec_list.shape)

10080
(256,)
(10080, 256)


In [6]:
# 模型保存
doc_model.save('opcode_doc.d2v')

In [7]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_list = to_categorical(label_list, 2)
X_train, X_test, y_train, y_test = train_test_split(doc_vec_list, label_list, 
                                                    random_state=2019, test_size = 0.2, stratify=label_list)
print(label_list)

/Users/devin/anaconda3/envs/python37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/devin/anaconda3/envs/python37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/devin/anaconda3/envs/python37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/devin/anaconda3/envs/python37/lib

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


/Users/devin/anaconda3/envs/python37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/devin/anaconda3/envs/python37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/devin/anaconda3/envs/python37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/devin/anacon

# SVM

In [7]:
from sklearn import svm

svc = svm.SVC(kernel='linear', C=1.0, gamma='auto', probability=True, random_state=None).fit(X_train_std, y_train)  # 线性核
# svc = svm.SVC(kernel='rbf', gamma=0.7, C=1.0).fit(X_train, y_train)  # 径向基核
# svc = svm.SVC(kernel='poly', degree=3, C=1.0, gamma='auto').fit(X_train, y_train)  # 多项式核
print('Finished training.\n');

Finished training.



In [8]:
from sklearn import metrics
predict_target = svc.predict(X_test_std)
print(metrics.classification_report(y_test, predict_target, 
                                    target_names=['Good sanitize', 'Bad sanitize']))

               precision    recall  f1-score   support

Good sanitize       0.62      0.75      0.68      1718
 Bad sanitize       0.55      0.39      0.46      1306

     accuracy                           0.60      3024
    macro avg       0.58      0.57      0.57      3024
 weighted avg       0.59      0.60      0.58      3024



# LSTM

In [10]:
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD

# 标准化
std_scaler = StandardScaler()
X_train_std = std_scaler.fit_transform(X_train).astype('float32')
X_test_std = std_scaler.transform(X_test).astype('float32')

X_train_std = X_train_std.reshape(-1, 4, 64)
print(X_train_std.shape)
# print(X_train_std)

batch = 48
timesteps = 4
data_dim = 64
units_size = 64
classes_num = 2

# Optimizer 自定义优化器
sgd1 = SGD(learning_rate=0.01)

model1 = Sequential()
model1.add(LSTM(units=units_size, dropout=0.25, recurrent_dropout=0.25,
                batch_input_shape=(batch, timesteps, data_dim),
                return_sequences=True, stateful=True,
                kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)
              )
         )
model1.add(LSTM(units_size,
                return_sequences=True, stateful=True))
model1.add(LSTM(units_size,
                stateful=True))
model1.add(Dropout(0.25))
model1.add(Dense(64, activation='relu'))
# model1.add(Dropout(0.2))
model1.add(Dense(classes_num, activation='softmax'))
model1.compile(loss='binary_crossentropy', optimizer=sgd1,
             metrics=['accuracy'])

(8064, 4, 64)


In [5]:
history1 = model1.fit(X_train_std, y_train, epochs=20)

NameError: name 'model1' is not defined

In [16]:
model1.save('doc2vec_lstm.h5')

# CNN

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adadelta, SGD, Nadam
from tensorflow.keras import regularizers

# 归一化
mm_scaler = MinMaxScaler()
X_train_std = mm_scaler.fit_transform(X_train).astype('float32')
X_test_std = mm_scaler.transform(X_test).astype('float32')
# # y_train = to_categorical(y_train, num_classes=2)
# # y_test = to_categorical(y_test, num_classes=2)
X_train_std = X_train_std.reshape(-1, 16, 16, 1)
batch_size = 64
epochs = 10

sgd2 = SGD(learning_rate=0.001)
nadam2 = Nadam(learning_rate=0.001)

model2 = Sequential()
model2.add(Conv2D(32, (5,5), activation='relu', input_shape=[16, 16, 1],
                  kernel_regularizer=regularizers.l2(0.001), 
                  activity_regularizer=regularizers.l1(0.001)
                 )
          )
model2.add(Conv2D(64, (5,5), activation='relu'))
model2.add(MaxPool2D(pool_size=(2,2)))
model2.add(Dropout(0.25))
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.25))
model2.add(Dense(2, activation='softmax'))

model2.compile(loss='cosine_proximity',
             optimizer=sgd2,
             metrics=['accuracy'])

In [48]:
hostory2 = model2.fit(X_train_std, y_train, batch_size=batch_size, epochs=10,
                      validation_split=0.2, verbose=1)

Train on 5644 samples, validate on 1412 samples
Epoch 1/10
5644/5644 [==============================] - 2s 393us/sample - loss: 0.6128 - acc: 1.0000 - val_loss: 0.6084 - val_acc: 1.0000
Epoch 2/10
5644/5644 [==============================] - 2s 385us/sample - loss: 0.6128 - acc: 1.0000 - val_loss: 0.6083 - val_acc: 1.0000
Epoch 3/10
5644/5644 [==============================] - 2s 384us/sample - loss: 0.6127 - acc: 1.0000 - val_loss: 0.6082 - val_acc: 1.0000
Epoch 4/10
5644/5644 [==============================] - 2s 379us/sample - loss: 0.6126 - acc: 1.0000 - val_loss: 0.6080 - val_acc: 1.0000
Epoch 5/10
5644/5644 [==============================] - 2s 383us/sample - loss: 0.6124 - acc: 1.0000 - val_loss: 0.6079 - val_acc: 1.0000
Epoch 6/10
5644/5644 [==============================] - 2s 384us/sample - loss: 0.6123 - acc: 1.0000 - val_loss: 0.6078 - val_acc: 1.0000
Epoch 7/10
5644/5644 [==============================] - 2s 386us/sample - loss: 0.6122 - acc: 1.0000 - val_loss: 0.6076 - va

In [49]:
model2.save('doc2vec_cov2d2.h5')

In [32]:
from keras.models import load_model
# model2 = load_model('doc2vec_cov2d.h5')
# X_test_std = X_test_std.reshape(-1, 16, 16, 1)
X_test_std = X_test_std.reshape(-1, 16, 16, 1)
loss, accuracy = model2.evaluate(X_test_std, y_test, verbose=1)
print('loss:%.4f accuracy:%.4f' %(loss, accuracy))


3024/3024 [==============================] - 0s 155us/sample - loss: 0.6127 - acc: 1.0000
loss:0.6127 accuracy:1.0000


In [52]:
sample2 = X_test_std.reshape(-1, 16, 16, 1)
pred_classes = model2.predict_classes(sample2)

right_count = 0
for y_pred, y_true in zip(pred_classes, y_test):
    print('%d %d' % (y_pred, y_true))

0 1
0 1
0 0
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 1
0 1
0 0
0 0
0 1
0 0
0 1
0 0
0 0
0 1
0 0
0 0
0 1
0 1
0 1
0 1
0 0
0 0
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 1
0 1
0 0
0 0
0 1
0 0
0 0
0 1
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 1
0 1
0 0
0 1
0 0
0 0
0 0
0 1
0 1
0 1
0 0
0 1
0 1
0 1
0 1
0 0
0 1
0 1
0 1
0 0
0 1
0 1
0 1
0 1
0 0
0 1
0 1
0 1
0 0
0 0
0 1
0 0
0 1
0 0
0 0
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 0
0 0
0 1
0 0
0 1
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 0
0 0
0 1
0 0
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 1
0 1
0 1
0 1
0 0
0 1
0 0
0 0
0 0
0 1
0 0
0 1
0 0
0 0
0 1
0 0
0 0
0 0
0 1
0 0
0 0
0 1
0 1
0 0
0 1
0 1
0 0
0 1
0 1
0 0
0 0
0 1
0 1
0 1
0 0
0 1
0 1
0 0
0 1
0 1
0 0
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 0
0 1
0 0
0 0
0 1
0 0
0 1
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 0
0 0
0 0
0 0
0 1
0 1
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 1
0 0
0 1
0 0
0 1
0 0
0 0
0 0
0 1
0 0


In [40]:
model2.evaluate?

Signature:
model2.evaluate(
    x=None,
    y=None,
    batch_size=None,
    verbose=1,
    sample_weight=None,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)
Docstring:
Returns the loss value & metrics values for the model in test mode.

Computation is done in batches.

Arguments:
    x: Input data. It could be:
      - A Numpy array (or array-like), or a list of arrays
        (in case the model has multiple inputs).
      - A TensorFlow tensor, or a list of tensors
        (in case the model has multiple inputs).
      - A dict mapping input names to the corresponding array/tensors,
        if the model has named inputs.
      - A `tf.data` dataset or a dataset iterator.
      - A generator or `keras.utils.Sequence` instance.
    y: Target data. Like the input data `x`,
      it could be either Numpy array(s) or TensorFlow tensor(s).
      It should be consistent with `x` (you cannot have Numpy inputs and
      tensor targe